In [1]:
# %%
import os
import nest_asyncio
import pandas as pd
import asyncio
import json
from typing import List, Sequence, Dict, Any

import openai
from toolhouse import Toolhouse
from llama_index.core.tools import FunctionTool, BaseTool
from llama_index.llms.openai import OpenAI
from llama_index.core.schema import NodeWithScore
from llama_index.core.workflow import Workflow, step, Context, StartEvent, StopEvent
from llama_index.core import (
    ServiceContext, SimpleDirectoryReader, Document, StorageContext, Prompt, GPTVectorStoreIndex,
    VectorStoreIndex, SummaryIndex
)
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.types import ChatMessage, MessageRole
from pydantic import BaseModel, Field

from pinecone import Pinecone, ServerlessSpec

from llama_parse import LlamaParse

from IPython.display import Markdown, display

from llama_index.agent.openai import OpenAIAgent

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings

# Apply nest_asyncio to allow nested event loops (useful in Jupyter notebooks)
nest_asyncio.apply()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from pinecone import ServerlessSpec

from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.tools import QueryEngineTool
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
toolhouse_api_key = os.getenv("TOOLHOUSE_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [4]:
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0)


In [5]:
# Defining the system prompt for the agent to reconcile column values
system_prompt_agent = (
    "You are an AI assistant that helps reconcile inconsistent car data values in a dataset.\n"
    "For each value, analyze the column-level context provided.\n"
    "If different terms refer to the same entity (e.g., 'Chevy' and 'Chevrolet'), correct the value to the standard form.\n"
    "So, in this dataset, for example, 'Chevy' and 'Chevrolet' refer to the same entity, so standardize all 'Chevy' values to 'Chevrolet'.\n"
    "Ensure the corrected value is consistent across all rows.\n"
    "Provide only the corrected value in JSON format without additional text.\n"
)

In [6]:
# %%
agent = OpenAIAgent.from_tools(
    llm=Settings.llm,
    verbose=True,
    system_prompt=system_prompt_agent,
    memory=None
    
)

In [7]:
def get_unique_values_by_index(dataset: pd.DataFrame, column_index: int):
    return dataset.iloc[:, column_index].unique()

In [8]:
def update_column_with_mapping(df: pd.DataFrame, col_idx: int, mapping: Dict[str, str]):
    # Map the values, but leave non-matching values unchanged
    df.iloc[:, col_idx] = df.iloc[:, col_idx].apply(lambda x: mapping.get(x, x))
    return df


In [9]:
# %%
dataset_with_nulls = pd.read_csv('car_data_with_synonyms.csv')
print("Dataset with null values:")
print(dataset_with_nulls.head(20))

Dataset with null values:
    id          brand               model  model_year    milage      fuel_type
0    0           MINI       Cooper S Base        2007  213000.0       Gasoline
1    1        Lincoln               LS V8        2003  143250.0       Gasoline
2    2          Chevy   Silverado 2500 LT        2002  136731.0  E85 Flex Fuel
3    3        Genesis    G90 5.0 Ultimate        2017   19500.0       Gasoline
4    4  Mercedes-Benz         Metris Base        2021    7388.0       Gasoline
5    5           Audi       A6 2.0T Sport        2018  213000.0       Gasoline
6    6           Audi           A8 L 3.0T        2024   62200.0       Gasoline
7    7      Chevrolet  Silverado 1500 1LZ        2016   20000.0  E85 Flex Fuel
8    8           Ford           F-150 XLT        2020   38352.0       Gasoline
9    9            BMW             M4 Base        2015   74850.0       Gasoline
10  10          Chevy   Silverado 1500 LT        2021   80000.0       Gasoline
11  11        Lincoln     

In [10]:
def enhance_values(agent, dataset: pd.DataFrame, column_index: int):
    uniq = get_unique_values_by_index(dataset, column_index)
    
    # Convert unique values into a string representation
    quote = list(map(lambda x: f"'{x}'", uniq))
    mk_str = ', '.join(quote)
    
    # Build the query string to send to the agent
    query_str = (
        f"{system_prompt_agent}\n"
        f"I have a dataset with the following unique values in a column: [{mk_str}]. "
        "Could you identify values that refer to the same entity and build a map to standardize them? "
        "Return only JSON mapping, no additional text."
    )
    
    # Reset the agent to avoid residual memory effects
    agent.reset()
    
    try:
        # Send the query to the agent and get the result
        result = agent.chat(query_str).response
        
        # Print the raw response to debug
        print(f"Raw agent response: {result}")
        
        # Clean the result by removing the ```json wrapper
        if result.startswith("```json"):
            result = result.strip("```json").strip("```")
        
        # Parse the result as JSON
        mapping = json.loads(result)
        print(f"Updating column '{column_index}' with mapping: {mapping}")
        
        # Apply the mapping to update the specified column
        dataset = update_column_with_mapping(dataset, column_index, mapping)
        
    except json.JSONDecodeError as e:
        print(f"Error parsing the agent response: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return dataset


In [14]:
completed_dataset = enhance_values(agent, dataset_with_nulls, column_index=2)


Added user message to memory: You are an AI assistant that helps reconcile inconsistent car data values in a dataset.
For each value, analyze the column-level context provided.
If different terms refer to the same entity (e.g., 'Chevy' and 'Chevrolet'), correct the value to the standard form.
So, in this dataset, for example, 'Chevy' and 'Chevrolet' refer to the same entity, so standardize all 'Chevy' values to 'Chevrolet'.
Ensure the corrected value is consistent across all rows.
Provide only the corrected value in JSON format without additional text.

I have a dataset with the following unique values in a column: ['Cooper S Base', 'LS V8', 'Silverado 2500 LT', 'G90 5.0 Ultimate', 'Metris Base', 'A6 2.0T Sport', 'A8 L 3.0T', 'Silverado 1500 1LZ', 'F-150 XLT', 'M4 Base', 'Silverado 1500 LT', 'Silverado 2500 HD', 'G80 3.8 Ultimate', 'GLA 250', 'F-150 XL', 'X5 xDrive40i', 'Cooper SE', 'Q7 3.0T Premium', 'G90 3.3T']. Could you identify values that refer to the same entity and build a map 

In [15]:
completed_dataset.to_csv('completed_car_data2.csv', index=False)
print("Completed dataset:")
print(completed_dataset.head(20))

Completed dataset:
    id          brand                         model  model_year    milage  \
0    0           MINI            Mini Cooper S Base        2007  213000.0   
1    1        Lincoln               Chevrolet LS V8        2003  143250.0   
2    2      Chevrolet   Chevrolet Silverado 2500 LT        2002  136731.0   
3    3        Genesis      Genesis G90 5.0 Ultimate        2017   19500.0   
4    4  Mercedes-Benz     Mercedes-Benz Metris Base        2021    7388.0   
5    5           Audi            Audi A6 2.0T Sport        2018  213000.0   
6    6           Audi                Audi A8 L 3.0T        2024   62200.0   
7    7      Chevrolet  Chevrolet Silverado 1500 1LZ        2016   20000.0   
8    8           Ford                Ford F-150 XLT        2020   38352.0   
9    9            BMW                   BMW M4 Base        2015   74850.0   
10  10      Chevrolet   Chevrolet Silverado 1500 LT        2021   80000.0   
11  11        Lincoln               Chevrolet LS V8      